In [1]:
pip install ipywidgets

You should consider upgrading via the '/Users/kehanbi/opt/anaconda3/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install lightfm

You should consider upgrading via the '/Users/kehanbi/opt/anaconda3/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd 
import json
import sys
import requests
import ipywidgets as widgets
from ipywidgets import interact,fixed
from IPython.display import display
import numpy as np
import os
import io
from io import StringIO

print(widgets.Button.on_click.__doc__)


Register a callback to execute when the button is clicked.

        The callback will be called with one argument, the clicked button
        widget instance.

        Parameters
        ----------
        remove: bool (optional)
            Set to true to remove the callback from the list of callbacks.
        


In [4]:
#upload dataset to Firebase Realtime database
spotify = pd.read_csv("SpotifyFeatures.csv", error_bad_lines=False, sep = ',')
#use data which the artist accurs more than 50 times
spotify = spotify.groupby('artist_name').filter(lambda x : len(x)>=50) 
df_artist = pd.DataFrame(spotify["artist_name"].unique())
df_artist = df_artist.reset_index()
df_artist = df_artist.rename(columns={'index':'artist_id', 0:'artist_name'})
df_artist
spotify  = pd.merge(spotify , df_artist, how='inner', on='artist_name')

jsondata = spotify.head(100000).to_json(orient = "records")
url = "https://final-project-551-8f1e5-default-rtdb.firebaseio.com/Spotify.json"
response = requests.put(url, jsondata)

print("-----Data Uploaded To Firebase Successful-----")

-----Data Uploaded To Firebase Successful-----


In [5]:
#upload metadata to firebase
url2 = "https://final-project-551-8f1e5-default-rtdb.firebaseio.com/Spotify/Metadata.json"
path = "SpotifyFeatures.csv"

fsize = os.path.getsize(path)
fsize = fsize/float(1024*1024)
fsize = str(round(fsize,2)) + 'MB'
ext = os.path.splitext(path)[-1]
rows_num = spotify.shape[0]
columns_num = spotify.shape[1]
features_num = len(spotify.columns.values)
features = spotify.columns.values

meta_dic = {"File_size":[fsize], "File_type": [ext], "Number of Rows": [rows_num], 
           "Number of columns" : [columns_num], "Number of features": [features_num], "Features": [features], 
            "Description" : ["Spotify music information such as artist, genre of music, track name, popularity, etc."]}

describe = pd.DataFrame(meta_dic)
describe = describe.to_json(orient = "records")
des = requests.put(url2, describe)

In [6]:
import numpy as np 
import pandas as pd 
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from scipy import sparse
import random
from sklearn.metrics.pairwise import cosine_similarity
import lightfm 
from lightfm import LightFM, cross_validation
from lightfm.evaluation import precision_at_k, auc_score

/Users/kehanbi/opt/anaconda3/lib/python3.8/site-packages/lightfm/_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(


In [7]:
url_model = "https://final-project-551-8f1e5-default-rtdb.firebaseio.com/Spotify.json"
data_model = requests.get(url_model)
reply_model = data_model.json()
del reply_model['Metadata']
spotify_firebase = pd.DataFrame(reply_model)
spotify_firebase = spotify_firebase.T

In [8]:
def create_interaction_matrix(df,user_col, item_col, rating_col, norm= False, threshold = None):
    interactions = df.groupby([user_col, item_col])[rating_col].sum().unstack().reset_index().fillna(0).set_index(user_col)
    if norm:
        interactions = interactions.applymap(lambda x: 1 if x > threshold else 0)
    return interactions

def create_user_dict(interactions):
    user_id = list(interactions.index)
    user_dict = {}
    counter = 0 
    for i in user_id:
        user_dict[i] = counter
        counter += 1
    return user_dict

def create_item_dict(df,id_col,name_col):
    item_dict ={}
    for i in range(df.shape[0]):
        item_dict[(df.loc[i,id_col])] = df.loc[i,name_col]
    return item_dict

def runMF(interactions, n_components=30, loss='warp', k=15, epoch=30,n_jobs = 4):
    model = LightFM(no_components= n_components, loss=loss,k=k)
    model.fit(x,epochs=epoch,num_threads = n_jobs)
    return model

#no_components – the dimensionality of the feature latent embeddings.
#loss – one of (‘logistic’, ‘bpr’, ‘warp’, ‘warp-kos’): the loss function.
    #WARP: Weighted Approximate-Rank Pairwise loss. Maximises the rank of positive examples by repeatedly 
    #sampling negative examples until rank violating one is found. Useful when only positive interactions are 
    #present and optimising the top of the recommendation list (precision@k) is desired.
#k – for k-OS training, the k-th positive example will be selected from the n positive examples sampled for every genre.

def sample_recommendation_user(model, interactions, actor_id, user_dict, 
                               item_dict,threshold = 0,nrec_items = 10, show = True):
    n_users, n_items = interactions.shape
    user_x = user_dict[actor_id]
    scores = pd.Series(model.predict(user_x,np.arange(n_items)))
    scores.index = interactions.columns
    scores = list(pd.Series(scores.sort_values(ascending=False).index))
    
    known_items = list(pd.Series(interactions.loc[actor_id,:][interactions.loc[actor_id,:] > threshold].index) \
                       .sort_values(ascending=False))
    
    scores = [x for x in scores if x not in known_items]
    return_score_list = scores[0:nrec_items]
    known_items = list(pd.Series(known_items).apply(lambda x: item_dict[x]))
    scores = list(pd.Series(return_score_list).apply(lambda x: item_dict[x]))
    if show == True:
        print("\n Recommended:")
        counter = 1
        for i in scores:
            print(str(counter) + '- ' + i)
            counter+=1
    return return_score_list


In [9]:
interactions = create_interaction_matrix(df = spotify_firebase, user_col = "genre", item_col = 'artist_id', 
                                         rating_col = 'popularity', norm= False, threshold = None)
interactions

artist_id,0,1,2,3,4,5,6,7,8,9,...,921,922,923,924,925,926,927,928,929,930
genre,,,,,,,,,,,,,,,,,,,,,
Alternative,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Anime,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Blues,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Children's Music,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Children’s Music,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Classical,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Comedy,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Country,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Dance,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
user_dict = create_user_dict(interactions=interactions)
artists_dict = create_item_dict(df = df_artist, id_col = 'artist_id', name_col = 'artist_name')
x = sparse.csr_matrix(interactions.values)
train, test = lightfm.cross_validation.random_train_test_split(x, test_percentage=0.2, random_state=None)


In [11]:
%time
model = runMF(interactions = train,
                 n_components = 30,
                 loss = 'warp',
                 k = 15,
                 epoch = 30,
                 n_jobs = 4)

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 3.1 µs


In [12]:
train_auc = auc_score(model, train, num_threads=4).mean()
print('Train AUC: %s' % train_auc)
test_auc = auc_score(model, test, train_interactions=train, num_threads=4).mean()
print('Test AUC: %s' % test_auc)
train_precision = precision_at_k(model, train, k=10).mean()
test_precision = precision_at_k(model, test, k=10, train_interactions=train).mean()
print('train Precision %.2f, test Precision %.2f.' % (train_precision, test_precision))

Train AUC: 0.9847761
Test AUC: 1.0
train Precision 0.77, test Precision 0.83.


In [13]:
# user can upload file here
def on_button_upload_file(b):
    with output1:
        output1.clear_output()
        uploaded_file = uploader.value
        print("Successful upload file", list(uploaded_file.keys())[0])
        #print(uploaded_file)
        metadata = list(uploaded_file.values())[0]['metadata']
        ext = os.path.splitext(metadata['name'])[-1]
        print("Metadata:\nFile type:",ext)
        fsize = metadata['size']
        fsize = fsize/float(1024*1024)
        print("File size:", str(round(fsize,2)), 'MB')
        file_name = os.path.splitext(metadata['name'])[0]
        #print(file_name)
        content = list(uploaded_file.values())[0]['content']
        describe = str(text_file_description.value)
        if ext == '.csv':  #csv file
            if len(str(text_split_symbol.value))==0:
                split = ";"
            else:
                split = str(text_split_symbol.value)
            upload_csv = pd.read_csv(io.BytesIO(content), sep = split)
            rows_num = upload_csv.shape[0]
            columns_num = upload_csv.shape[1]
            features_num = len(upload_csv.columns)
            features = upload_csv.columns.values
            print(features)
            print(type(features))
            print("There are {} rows and {} columns in dataset.".format(rows_num, columns_num))
            print("There are {} features in dataset".format(features_num))
            print("The features and their datatypes:")
            for i in range(len(upload_csv.columns)):
                print(" |-" + upload_csv.columns[i] + " - " + str(upload_csv.dtypes[i]))
                
            #upload dataset to firebase
            url_upload_csv = "https://final-project-551-8f1e5-default-rtdb.firebaseio.com/" \
                                                    + str(file_name) + ".json"
            csv_to_json = upload_csv.to_json(orient = "records")
            #print(csv_to_json)
            response = requests.put(url_upload_csv, csv_to_json)
            print("{} has been uploaded to Firebase.".format(list(uploaded_file.keys())[0]))
            
            #upload metadata
            url_upload_csv_metadata = "https://final-project-551-8f1e5-default-rtdb.firebaseio.com/" \
                                                    + str(file_name) + "/Metadata.json"
            meta_dic_csv = {"File_size":[fsize], "File_type": [ext], "Number of Rows": [rows_num], 
                       "Number of columns" : [columns_num], "Number of features": [features_num], 
                            "Features": [features], "Description" : [describe]}
            describe_csv = pd.DataFrame(meta_dic_csv)
            describe_csv = describe_csv.to_json(orient = "records")
            des = requests.put(url_upload_csv_metadata, describe_csv)
        elif ext == '.json':  #json file
            json_content = json.loads(str(content,'utf-8'))
            rows_num_json = len(json_content)
            json_feature = list(json_content[0].keys())
            features_num_json = len(json_feature)
            
            print("There are {} rows and {} columns in dataset.".format(rows_num_json, features_num_json))
            print("There are {} features in dataset".format(features_num_json))
            print("The features and their datatypes:")
            for key, value in json_content[0].items():
                print(" |-" , key, " - ", str(type(value)))

            #upload dataset to firebase
            upload_json = str(content,'utf-8')
            url_upload_json = "https://final-project-551-8f1e5-default-rtdb.firebaseio.com/" \
                                                    + str(file_name) + ".json"
            response_json = requests.put(url_upload_json, upload_json)
            print("{} has been uploaded to Firebase.".format(list(uploaded_file.keys())[0]))
            
            #upload metadata to firebase
            url_upload_json_metadata = "https://final-project-551-8f1e5-default-rtdb.firebaseio.com/" \
                                                    + str(file_name) + "/Metadata.json"
            meta_dic_json = {"File_size":[fsize], "File_type": [ext], "Number of Rows": [rows_num_json], 
                       "Number of columns" : [features_num_json], "Number of features": [features_num_json], 
                             "Features": [json_feature], "Description" : [describe]}
            describe_json = pd.DataFrame(meta_dic_json)
            describe_json = describe_json.to_json(orient = "records")
            des = requests.put(url_upload_json_metadata, describe_json) 
        else:
            print("The file type is uncorrect, please upload other file.")

In [14]:
print("Please upload .csv or .json files here.")
print("Type in the special split symbol of .csv files here, e.g. ',';'\\t'; etc. If the split symbol of your file is ';', keep blank.")
output1 = widgets.Output()
uploader = widgets.FileUpload(
    accept='.csv,.json',  # Accepted file extension e.g. '.txt', '.pdf', 'image/*', 'image/*,.pdf'
    multiple=True  # True to accept multiple files upload else False
)
button = widgets.Button(
    description='Upload',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me2',
    icon='check' # (FontAwesome names without the `fa-` prefix)
)
text_split_symbol = widgets.Text(
    value='',
    placeholder='Type in the special split symbol',
    description='Split symbol:',
    disabled=False
)

text_file_description = widgets.Text(
    value='',
    placeholder='Describe your file in short sentence',
    description='Description:',
    disabled=False
)

button.on_click(on_button_upload_file)
left_box = widgets.VBox([text_split_symbol, text_file_description])
right_box = widgets.VBox([uploader, button])
hbox = widgets.HBox([left_box, right_box])
display(hbox, output1)


Please upload .csv or .json files here.
Type in the special split symbol of .csv files here, e.g. ',';'\t'; etc. If the split symbol of your file is ';', keep blank.


Output()

In [15]:
url_get_data = "https://final-project-551-8f1e5-default-rtdb.firebaseio.com/.json" 
get_data_response = requests.get(url_get_data)
reply = get_data_response.json()
dataset_dic = reply.keys()
dataset_dic = list(dataset_dic)

In [16]:
output = widgets.Output()

dslist = []
for i in dataset_dic:
    tup = (i,i)
    dslist.append(tup)

view_data = {}
def on_button_exploring_clicked(b):
    with output:
        output.clear_output()
        view_data = reply[dropdown_ds.value]
        meta = pd.DataFrame(view_data["Metadata"])
        #print(meta)
        df2 = view_data.copy()
        del df2['Metadata']
        line_data = pd.DataFrame(df2)
        if b.description == 'View Dataset':
            print("Dataset: {}".format(dropdown_ds.value))
            print("Description:", view_data["Metadata"][0]['Description'])
        elif b.description == 'Metadata':
            print(meta.T)
            print("\nFeatures are: ")
            for i in list(meta["Features"])[0]:
                print("|-", i)
            #, list(meta["Features"])[0])
            #print("Dataset has {} rows and {} columns\nNumber of features = {}\nFeatures are: {}".format(valdic[0][1].shape[0], valdic[0][1].shape[1], len(valdic[0][1].columns.values), valdic[0][1].columns.values))
        elif b.description == 'First 10 lines':
            print(line_data.head(10).T)
            #print(valdic[0][1].head(10))
        elif b.description == 'Last 10 lines':
            print(line_data.tail(10).T)
            #print(valdic[0][1].tail(10))
        else:    
            print(line_data.sample(n=2).T)

In [17]:
dropdown_ds = widgets.Dropdown(
    options = dslist,
    value = "Spotify",
    description='Dataset:',
)
button_exploring = widgets.Button(
    description='View Dataset',
    disabled=False,
    button_style='', 
    tooltip='Click me2',
    icon='check' 
)
button_metadata = widgets.Button(
    description='Metadata',
    disabled=False,
    button_style='',
)
button_head10 = widgets.Button(
    description='First 10 lines',
    disabled=False,
    button_style='', 
)
button_tail10 = widgets.Button(
    description='Last 10 lines',
    disabled=False,
    button_style='', 
)
button_rand10 = widgets.Button(
    description='Random 10 lines',
    disabled=False,
    button_style='', 
)
radiobuttons = widgets.RadioButtons(
    options=['First 10 lines', 'Last 10 lines', 'Random 10 lines'],
    description='Preview of data:',
    disabled=False
)

button_exploring.on_click(on_button_exploring_clicked)
button_metadata.on_click(on_button_exploring_clicked)
button_head10.on_click(on_button_exploring_clicked)
button_tail10.on_click(on_button_exploring_clicked)
button_rand10.on_click(on_button_exploring_clicked)

left_box = widgets.VBox([dropdown_ds])
right_box = widgets.VBox([button_exploring])
hbox1 = widgets.HBox([left_box, right_box])
hbox2 = widgets.HBox([button_metadata, button_head10,button_tail10,button_rand10])
display(hbox1,hbox2,output)

Output()

In [18]:
def  on_button_ml_model(b): 
    with output2:
        output2.clear_output()
        if b.description == 'Upload':
            rec_list = sample_recommendation_user(model = model, 
                                      interactions = interactions, 
                                      actor_id = text_song_type.value, 
                                      user_dict = user_dict,
                                      item_dict = artists_dict, 
                                      threshold = 0,
                                      nrec_items = 10,
                                      show = True)

In [19]:
output2 = widgets.Output()
button = widgets.Button(
    description='Upload',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me2',
    icon='check' # (FontAwesome names without the `fa-` prefix)
)

text_song_type = widgets.Text(
    value='',
    placeholder='Type in music type',
    description='Type:',
    disabled=False
)
text_song_type.value=''

button.on_click(on_button_ml_model)

left_box = widgets.VBox([text_song_type])
right_box = widgets.VBox([button])
hbox = widgets.HBox([left_box, right_box])
display(hbox, output2)

Output()